In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [29]:
paper_pmid = 29187527
paper_name = 'eisenberg_bord_bohnert_2018' 

In [30]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [31]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

### Part 1

In [32]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, names=['genes','data'], sep='\t')

In [33]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 22 x 2


In [34]:
original_data['genes'] = original_data['genes'].astype(str)

In [35]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [36]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['genes'], to='orf')

In [37]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, data, orfs]
Index: []


In [38]:
original_data.set_index('orfs', inplace=True)

### Part 2

In [39]:
original_data2 = pd.read_excel('raw_data/JCB_201704122_TableS1.xlsx', sheet_name='HITS from screens', skiprows=2, nrows=30)

In [40]:
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 30 x 16


In [41]:
original_data2['ORF'] = original_data2['ORF'].astype(str)

In [42]:
# Eliminate all white spaces & capitalize
original_data2['ORF'] = clean_orf(original_data2['ORF'])

In [ ]:
# Translate to ORFs 
original_data2['ORF'] = translate_sc(original_data2['ORF'], to='orf')

In [ ]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['ORF'])
print(original_data2.loc[~t,])

In [ ]:
original_data2['data'] = 1

In [ ]:
original_data2.set_index('ORF', inplace=True)

### Merge

In [ ]:
data = original_data[['data']].join(original_data2[['data']], lsuffix='_lower', rsuffix='_higher', how='outer')

In [ ]:
data['data'] = data[['data_lower','data_higher']].mean(axis=1)

In [ ]:
data = data[['data']]

### Remove essential genes (if any) -- original data included deletions and DAmP strains

In [ ]:
ess = ~is_essential(data.index.values)

In [26]:
data.shape

(52, 1)

In [27]:
data = data.loc[ess.values,:]

In [28]:
data.shape

(7, 1)

# Prepare the final dataset

In [58]:
dataset_ids = [16438]

In [59]:
datasets = datasets.reindex(index=dataset_ids)

In [60]:
data.columns = datasets['name'].values

In [61]:
data = data.groupby(data.index).mean()

In [62]:
# Create row index
data.index.name='orf'

In [63]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 52 x 1


# Print out

In [65]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [66]:
from IO.save_data_to_db2 import *

In [67]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [68]:
save_data_to_db(data, paper_pmid)